# Policy Checker with Azure AI Search

This notebook demonstrates how to use agent operations with the Azure AI Search tool from the Azure Agents service using a synchronous client.

## Setup and Imports

In [5]:
# Import necessary libraries and modules
import os
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential, AzureCliCredential, ChainedTokenCredential
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import AzureAISearchQueryType, AzureAISearchTool, ListSortOrder, MessageRole

# Load environment variables from .env file
load_dotenv(dotenv_path='../.env', override=True)  # This forces a reload of the .env file

print("✅ Libraries imported and environment variables loaded")

# Debug: Check if critical environment variables are loaded
required_vars = ['AI_FOUNDRY_PROJECT_ENDPOINT', 'MODEL_DEPLOYMENT_NAME', 'AZURE_AI_CONNECTION_ID']
for var in required_vars:
    value = os.environ.get(var)
    if value:
        print(f"✅ {var}: Found")
    else:
        print(f"❌ {var}: Not found")

print("\n🔍 Setting up Azure authentication...")

✅ Libraries imported and environment variables loaded
✅ AI_FOUNDRY_PROJECT_ENDPOINT: Found
✅ MODEL_DEPLOYMENT_NAME: Found
✅ AZURE_AI_CONNECTION_ID: Found

🔍 Setting up Azure authentication...


## Configuration

In [9]:
# Retrieve endpoint and model deployment name from environment variables
project_endpoint = os.environ.get("AI_FOUNDRY_PROJECT_ENDPOINT")  # Ensure the PROJECT_ENDPOINT environment variable is set
model_deployment_name = os.environ.get("MODEL_DEPLOYMENT_NAME")  # Ensure the MODEL_DEPLOYMENT_NAME environment variable is set
sc_connection_id = os.environ.get("AZURE_AI_CONNECTION_ID")  # Ensure the AZURE_AI_CONNECTION_ID environment variable is set

# Enhanced Azure authentication with fallback options
print("🔐 Attempting Azure authentication...")

# Try multiple authentication methods in order of preference
try:
    # First, try Azure CLI credentials (if you're signed in with az login)
    cli_credential = AzureCliCredential()
    
    # Create a chained credential that falls back through multiple options
    credential = ChainedTokenCredential(
        cli_credential,  # Try Azure CLI first
        DefaultAzureCredential(exclude_interactive_browser_credential=False)  # Then default options
    )
    
    print("✅ Authentication credential created successfully")
    
except Exception as e:
    print(f"❌ Authentication setup failed: {e}")
    print("\n💡 To fix this:")
    print("1. Run 'az login' in your terminal")
    print("2. Or sign in to Azure through VS Code (Azure extension)")
    raise

# Verify we have all required environment variables
if not all([project_endpoint, model_deployment_name, sc_connection_id]):
    missing = []
    if not project_endpoint: missing.append("AI_FOUNDRY_PROJECT_ENDPOINT")
    if not model_deployment_name: missing.append("MODEL_DEPLOYMENT_NAME")
    if not sc_connection_id: missing.append("AZURE_AI_CONNECTION_ID")
    
    print(f"❌ Missing required environment variables: {missing}")
    print("Please check your .env file and ensure these variables are set.")
    raise ValueError(f"Missing environment variables: {missing}")

print(f"✅ All required environment variables loaded")
print(f"📍 Project endpoint: {project_endpoint[:50]}...")

project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=credential,  # Use our enhanced credential
)

print("✅ AIProjectClient created successfully")

with project_client:
    print("🔍 Initializing Azure AI Search tool...")
    
    # Initialize the Azure AI Search tool with the required parameters
    ai_search = AzureAISearchTool(
        index_connection_id=sc_connection_id,  # Ensure the AZURE_AI_CONNECTION_ID environment variable is set
        index_name="insurance-documents-index",  # Name of the search index
        query_type=AzureAISearchQueryType.SIMPLE,  # Query type (e.g., SIMPLE, FULL)
        top_k=3,  # Number of top results to retrieve
        filter="",  # Optional filter for search results
    )
    
    print("✅ Azure AI Search tool initialized")
    print("🤖 Creating insurance policy checker agent...")

    # Create an agent with the specified model, name, instructions, and tools
    agent = project_client.agents.create_agent(
        model=model_deployment_name,  # Model deployment name
        name="policy-checker",  # Name of the agent
        instructions=
        """"
        You are an expert Insurance Policy Checker Agent specialized in analyzing auto insurance policies and validating claim coverage. Your primary responsibilities include:

        **Core Functions:**
        - Analyze insurance policy documents to determine coverage details
        - Validate if specific claims are covered under policy terms
        - Explain policy limits, deductibles, and exclusions
        - Identify coverage gaps or restrictions
        - Provide clear explanations of policy benefits

        **Policy Types You Handle:**
        - Commercial Auto Policies
        - Comprehensive Auto Policies  
        - High Value Vehicle Policies
        - Liability Only Policies
        - Motorcycle Policies

        **Analysis Guidelines:**
        1. Always reference specific policy sections when providing coverage determinations
        2. Clearly state coverage limits, deductibles, and any applicable restrictions
        3. Identify any exclusions that may apply to the claim
        4. Be precise about effective dates and policy periods
        5. Flag any discrepancies between claim details and policy terms

        **Response Format:**
        - Start with a clear coverage determination (COVERED/NOT COVERED/PARTIAL COVERAGE)
        - Provide the specific policy section reference
        - Explain coverage limits and deductibles
        - List any relevant exclusions or conditions
        - Suggest next steps if coverage issues exist
        - Everything in a clear, concise manner in one paragraph.

        **Tone:** Professional, accurate, and helpful. Always be thorough in your analysis while remaining clear and concise.

        When you cannot find specific information in the policy documents, clearly state what information is missing and what additional documentation would be needed.
        """,  # Instructions for the agent
        tools=ai_search.definitions,  # Tools available to the agent
        tool_resources=ai_search.resources,  # Resources for the tools
    )
    print(f"✅ Created agent, ID: {agent.id}")

    # Create a thread for communication with the agent
    thread = project_client.agents.threads.create()
    print(f"✅ Created thread, ID: {thread.id}")

    print("💬 Sending question to the agent...")
    # Send a message to the thread
    message = project_client.agents.messages.create(
        thread_id=thread.id,  # ID of the thread
        role=MessageRole.USER,  # Role of the message sender (e.g., user)
        content="What motorcycle coverage options are available?",  # Message content
    )
    print(f"✅ Created message, ID: {message['id']}")

    print("🚀 Processing agent response...")
    # Create and process an agent run in the thread using the tools
    run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
    print(f"✅ Run finished with status: {run.status}")

    if run.status == "failed":
        # Log the error if the run fails
        print(f"❌ Run failed: {run.last_error}")
    else:
        print("📝 Agent Response:")
        print("=" * 80)

    # Fetch and log all messages from the thread
    messages = project_client.agents.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
    for message in messages:
        # Only show the agent's response, not the user's question
        if message.role == MessageRole.AGENT:
            if message.content and len(message.content) > 0:
                content_item = message.content[0]
                if content_item.get('type') == 'text' and 'text' in content_item:
                    agent_response = content_item['text']['value']
                    print(agent_response)
                    break  # Only show the first agent response

🔐 Attempting Azure authentication...
✅ Authentication credential created successfully
✅ All required environment variables loaded
📍 Project endpoint: https://msagthack-aifoundry-anzlgneac4wte.services...
✅ AIProjectClient created successfully
🔍 Initializing Azure AI Search tool...
✅ Azure AI Search tool initialized
🤖 Creating insurance policy checker agent...
✅ Created agent, ID: asst_W8yVb2or8rln7uIsEBQKn2FR
✅ Created agent, ID: asst_W8yVb2or8rln7uIsEBQKn2FR
✅ Created thread, ID: thread_vTJuhY9lm9C1BEJuJk0YP5R1
💬 Sending question to the agent...
✅ Created thread, ID: thread_vTJuhY9lm9C1BEJuJk0YP5R1
💬 Sending question to the agent...
✅ Created message, ID: msg_WrjRWPuLg6PYeVxIQdNNxEKg
🚀 Processing agent response...
✅ Created message, ID: msg_WrjRWPuLg6PYeVxIQdNNxEKg
🚀 Processing agent response...
✅ Run finished with status: RunStatus.COMPLETED
📝 Agent Response:
✅ Run finished with status: RunStatus.COMPLETED
📝 Agent Response:
COVERED: The motorcycle insurance policy offers a range of c

Amazing! So what you have done so far is creating an agent, that consults the Azure AI Search Index that searches on your policies databases. Your query has made it such as it replies to your query with a text based solution. Let's print it out!

In [8]:

# Let's add some comprehensive error checking and debugging
print("🔍 Comprehensive System Check:")
print("=" * 50)

# 1. Check Azure CLI authentication
try:
    import subprocess
    result = subprocess.run(['az', 'account', 'show'], capture_output=True, text=True)
    if result.returncode == 0:
        print("✅ Azure CLI: Successfully authenticated")
        # Parse the account info
        import json
        account_info = json.loads(result.stdout)
        print(f"   📋 Account: {account_info.get('user', {}).get('name', 'Unknown')}")
        print(f"   🏢 Subscription: {account_info.get('name', 'Unknown')}")
    else:
        print("❌ Azure CLI: Not authenticated")
        print(f"   Error: {result.stderr}")
except Exception as e:
    print(f"⚠️ Azure CLI check failed: {e}")

# 2. Test credential authentication
print("\n🔐 Testing Azure Credentials:")
try:
    from azure.identity import AzureCliCredential
    cli_cred = AzureCliCredential()
    # Try to get a token to test the credential
    token = cli_cred.get_token("https://management.azure.com/.default")
    print("✅ AzureCliCredential: Working")
    print(f"   🎫 Token expires: {token.expires_on}")
except Exception as e:
    print(f"❌ AzureCliCredential failed: {e}")

# 3. Test environment variables again
print("\n🌍 Environment Variables Check:")
env_vars = {
    'AI_FOUNDRY_PROJECT_ENDPOINT': os.environ.get('AI_FOUNDRY_PROJECT_ENDPOINT'),
    'MODEL_DEPLOYMENT_NAME': os.environ.get('MODEL_DEPLOYMENT_NAME'), 
    'AZURE_AI_CONNECTION_ID': os.environ.get('AZURE_AI_CONNECTION_ID')
}

for var_name, var_value in env_vars.items():
    if var_value:
        # Show first and last few characters for security
        masked_value = f"{var_value[:10]}...{var_value[-10:]}" if len(var_value) > 20 else var_value
        print(f"✅ {var_name}: {masked_value}")
    else:
        print(f"❌ {var_name}: Missing")

# 4. Test AIProjectClient creation
print("\n🚀 Testing AIProjectClient Connection:")
try:
    from azure.identity import ChainedTokenCredential, AzureCliCredential, DefaultAzureCredential
    
    # Create credential chain
    credential = ChainedTokenCredential(
        AzureCliCredential(),
        DefaultAzureCredential(exclude_interactive_browser_credential=False)
    )
    
    project_endpoint = os.environ.get("AI_FOUNDRY_PROJECT_ENDPOINT")
    if not project_endpoint:
        raise ValueError("AI_FOUNDRY_PROJECT_ENDPOINT not found")
    
    # Test client creation
    test_client = AIProjectClient(
        endpoint=project_endpoint,
        credential=credential
    )
    print("✅ AIProjectClient: Created successfully")
    
    # Test a simple operation (like listing models)
    try:
        with test_client:
            print("✅ AIProjectClient: Connection established")
    except Exception as conn_e:
        print(f"⚠️ AIProjectClient: Created but connection test failed: {conn_e}")
        
except Exception as e:
    print(f"❌ AIProjectClient creation failed: {e}")
    print("   This indicates authentication or endpoint issues")

print("\n" + "=" * 50)

🔍 Comprehensive System Check:
⚠️ Azure CLI check failed: [WinError 2] The system cannot find the file specified

🔐 Testing Azure Credentials:
✅ AzureCliCredential: Working
   🎫 Token expires: 1760968944

🌍 Environment Variables Check:
✅ AI_FOUNDRY_PROJECT_ENDPOINT: https://ms...lgneac4wte
✅ MODEL_DEPLOYMENT_NAME: gpt-4.1-mini
✅ AZURE_AI_CONNECTION_ID: /subscript...e-aisearch

🚀 Testing AIProjectClient Connection:
✅ AIProjectClient: Created successfully
✅ AIProjectClient: Connection established

✅ AzureCliCredential: Working
   🎫 Token expires: 1760968944

🌍 Environment Variables Check:
✅ AI_FOUNDRY_PROJECT_ENDPOINT: https://ms...lgneac4wte
✅ MODEL_DEPLOYMENT_NAME: gpt-4.1-mini
✅ AZURE_AI_CONNECTION_ID: /subscript...e-aisearch

🚀 Testing AIProjectClient Connection:
✅ AIProjectClient: Created successfully
✅ AIProjectClient: Connection established



## Error Analysis and Troubleshooting

Run the cell above to get a comprehensive system check. Based on the results, we can determine:

### 🔍 **What to Look For:**

1. **Azure CLI Status** - Should show "Successfully authenticated"
2. **Credential Test** - AzureCliCredential should be working
3. **Environment Variables** - All three required variables should be present
4. **AIProjectClient** - Should create and connect successfully

### ⚠️ **Common Issues and Solutions:**

| Issue | Symptoms | Solution |
|-------|----------|----------|
| **Authentication Failed** | ❌ Azure CLI not authenticated | Run `az login` in terminal |
| **Missing Env Variables** | ❌ Variables not found | Check `.env` file path and content |
| **Connection Failed** | ⚠️ Client created but connection test failed | Check endpoint URL and network |
| **Permission Issues** | ❌ Access denied errors | Verify Azure permissions for the subscription |

### 🚨 **Critical vs Non-Critical Errors:**

- **CRITICAL**: Authentication failures, missing endpoints - Must be fixed
- **NON-CRITICAL**: Warning messages, minor timeout issues - Can often be ignored
- **INFORMATIONAL**: Debug messages, progress updates - Safe to ignore

## 🚀 Success! Your Policy Checker Agent is Working!

Your agent successfully analyzed the motorcycle coverage question by:
1. **Searching the insurance policy database** using Azure AI Search
2. **Finding relevant policy sections** (MOTO-001, Sections 1 & 2) 
3. **Providing expert analysis** with coverage details, exclusions, and next steps
4. **Referencing specific policy documents** for verification

The agent demonstrated it can handle complex insurance queries and provide professional, detailed responses!

In [ ]:
# 🧪 Test Your Policy Checker with Different Questions!
# Change the question below and run this cell to test different insurance scenarios

def ask_policy_question(question):
    """Ask the policy checker agent a new question"""
    try:
        # Recreate the client connection since the previous one was closed
        project_client_new = AIProjectClient(
            endpoint=project_endpoint,
            credential=credential,
        )
        
        with project_client_new:
            # Create a new thread for this question
            new_thread = project_client_new.agents.threads.create()
            print(f"🆕 Created new thread: {new_thread.id}")
            
            # Send the question
            message = project_client_new.agents.messages.create(
                thread_id=new_thread.id,
                role=MessageRole.USER,
                content=question
            )
            print(f"❓ Question: {question}")
            print("🤔 Agent is thinking...")
            
            # Process the response
            run = project_client_new.agents.runs.create_and_process(
                thread_id=new_thread.id, 
                agent_id=agent.id
            )
            
            if run.status == "completed":
                # Get the response
                messages = project_client_new.agents.messages.list(
                    thread_id=new_thread.id, 
                    order=ListSortOrder.ASCENDING
                )
                
                for message in messages:
                    if message.role == MessageRole.AGENT:
                        if message.content and len(message.content) > 0:
                            content_item = message.content[0]
                            if content_item.get('type') == 'text' and 'text' in content_item:
                                response = content_item['text']['value']
                                print(f"\n💡 Agent Response:")
                                print("=" * 80)
                                print(response)
                                return response
            else:
                print(f"❌ Query failed with status: {run.status}")
                if hasattr(run, 'last_error'):
                    print(f"Error: {run.last_error}")
                
    except Exception as e:
        print(f"❌ Error asking question: {e}")
        return None

# 🎯 Try different questions! Modify the question below:
print("🚀 Testing Policy Checker with a new question...")
print("=" * 60)

# Change this question to test different scenarios:
test_question = "What are the coverage limits for collision damage on a commercial auto policy?"

# Ask the question
ask_policy_question(test_question)

🚀 Testing Policy Checker with a new question...
❌ Error asking question: HTTP transport has already been closed. You may check if you're calling a function outside of the `with` of your client creation, or if you called `close()` on your client already.
